In [238]:
import networkx as nx
import matplotlib.pyplot as plt
from ortools.sat.python import cp_model

In [239]:
teachers = {
    "Alice" : {
        "class1" : {"English", "Bengali"},
        "class2" : {"H. Math"},
        "class3" : {}
    },
    "Bob" : {
        "class1" : {"Math"},
        "class2" : {"Math"},
        "class3" : {}
    },
    "Jolly" : {
        "class1" : {},
        "class2" : {},
        "class3" : {"ICT", "Biology"}
    },
    "Mark" : {
        "class1" : {},
        "class2" : {"Physics"},
        "class3" : {"BGS"}
    },
}

school_data = {
    "class1": {"Math": 6, "English": 6, "Bengali": 6},
    "class2": {"Physics": 3, "H. Math": 2, "Math": 6},
    "class3": {"Biology": 3, "BGS": 3, "ICT": 3}
}
num_days = 6
num_shifts = 3

In [240]:
model = cp_model.CpModel()

In [241]:
schedule = {}
for teacher in teachers:
    for clss, subjects in school_data.items():
        for subject, days in subjects.items():
            if subject in teachers[teacher].get(clss, []): #Ensuring the teacher is qualified to teach the subject
                for day in range(num_days):
                    for shift in range(num_shifts):
                        schedule[(teacher, clss, subject, day, shift)] = model.NewBoolVar(f"{teacher}_{clss}_{subject}_{day}_{shift}")

In [242]:
#1st Constraint
for teacher in teachers:
    for clss, subjects in school_data.items():
        for subject, required_days in subjects.items():
            if subject in teachers[teacher].get(clss, []):  # Ensuring the teacher is qualified to teach the subject
                # Sum of all shifts across all days for this subject must equal the required days
                model.Add(sum(schedule[(teacher, clss, subject, day, shift)] 
                              for day in range(num_days) 
                              for shift in range(num_shifts)) == required_days)

In [243]:
#2nd Constraint
for teacher in teachers:
    for clss, subjects in school_data.items():
        for subject, required_days in subjects.items():
            if subject not in teachers[teacher].get(clss, []):  # Teacher isn't qualified for this subject
                model.Add(sum(schedule.get((teacher, clss, subject, day, shift), 0) 
                              for day in range(num_days) 
                              for shift in range(num_shifts)) == 0)

In [244]:
# 3rd Constraint
for teacher in teachers:
    for day in range(num_days):
        for shift in range(num_shifts):
            model.Add(sum(schedule.get((teacher, clss, subject, day, shift), 0) 
                        for clss in school_data
                        for subject in school_data[clss]) <= 1) # teacher can't teach more than one subject at the same time

In [245]:
# for day in range(num_days):
#     for shift in range(num_shifts):
#         model.Add(sum(schedule.get((teacher, clss, subject, day, shift), 0) 
#                     for teacher in teachers
#                     for clss in school_data
#                     for subject in school_data[clss]) <= 1) # multiple teacher can't teach a class in the same shift simultaneously

In [246]:
# 4th Constraint
for clss in school_data:
    for day in range(num_days):
        for shift in range(num_shifts):
            model.Add(sum(schedule.get((teacher, clss, subject, day, shift), 0) 
                          for teacher in teachers 
                          for subject in school_data[clss]) <= 1) # multiple teacher can't teach a class in the same shift simultaneously

In [247]:
# 5th Constraint
for clss in school_data:
    for subject in school_data[clss]:
        for day in range(num_days):
                model.Add(sum(schedule.get((teacher, clss, subject, day, shift), 0) 
                              for teacher in teachers 
                              for shift in range(num_shifts)) <= 1) # same subject can't taught multiple times in the same day

In [255]:
# 6th constraint
for clss in school_data:
    for subject, required_days in school_data[clss].items():
        model.Add(
            sum(
                schedule.get((teacher, clss, subject, day, shift), 0)
                for teacher in teachers 
                for day in range(num_days)
                for shift in range(num_shifts)
            ) <= required_days
        )


In [250]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [251]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for teacher in teachers:
        print(f"\nSchedule for {teacher}:")
        for key, value_var in schedule.items():  # Iterate through schedule items
            if solver.Value(value_var) == 1:
                t, cls, subject, day, shift = key
                if t == teacher:  # Filter for the current teacher
                    print(f"  Day {day}, Shift {shift}: {cls} - {subject}")
else:
    print("No solution found!")


Schedule for Alice:
  Day 0, Shift 1: class1 - English
  Day 1, Shift 2: class1 - English
  Day 2, Shift 1: class1 - English
  Day 3, Shift 2: class1 - English
  Day 4, Shift 2: class1 - English
  Day 5, Shift 0: class1 - English
  Day 0, Shift 2: class1 - Bengali
  Day 1, Shift 0: class1 - Bengali
  Day 2, Shift 2: class1 - Bengali
  Day 3, Shift 1: class1 - Bengali
  Day 4, Shift 0: class1 - Bengali
  Day 5, Shift 2: class1 - Bengali
  Day 0, Shift 0: class2 - H. Math
  Day 4, Shift 1: class2 - H. Math

Schedule for Bob:
  Day 0, Shift 0: class1 - Math
  Day 1, Shift 1: class1 - Math
  Day 2, Shift 0: class1 - Math
  Day 3, Shift 0: class1 - Math
  Day 4, Shift 1: class1 - Math
  Day 5, Shift 1: class1 - Math
  Day 0, Shift 1: class2 - Math
  Day 1, Shift 0: class2 - Math
  Day 2, Shift 2: class2 - Math
  Day 3, Shift 1: class2 - Math
  Day 4, Shift 2: class2 - Math
  Day 5, Shift 0: class2 - Math

Schedule for Jolly:
  Day 0, Shift 0: class3 - Biology
  Day 1, Shift 1: class3 - Bio

In [252]:
# day_names = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Tuesday", "Thursday"]
# if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
#     with open("schedule_results.csv", mode="w", newline="") as file:
#         writer = csv.writer(file)
#         writer.writerow(["", "Class", "Shift", "Day", "Subject"])  # Header row

#         for teacher in teachers:
#             for key, value_var in schedule.items():
#                 if solver.Value(value_var) == 1:
#                     t, cls, subject, day, shift = key
#                     if teacher == t:
#                         writer.writerow([t, cls, shift, day_names[day], subject])  # Write to CSV
# else:
#     print("No solution found!")

In [253]:
import csv
from ortools.sat.python import cp_model

# Example day_names (be sure to match your actual days)
day_names = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # 1. Gather all assignments where solver.Value(...) == 1
    results = []
    for teacher in teachers:
        for key, value_var in schedule.items():
            if solver.Value(value_var) == 1:
                t, cls, subject, day, shift = key
                if t == teacher:
                    # Store (teacher, class, shift, day_index, day_name, subject)
                    results.append((t, cls, shift, day, day_names[day], subject))

    # 2. Sort results so they appear in a logical order
    #    e.g., by day, then by class, then by shift
    results.sort(key=lambda x: (x[3], x[1], x[2]))

    # 3. Write to CSV with "grouped" days
    with open("schedule_results.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        # Header row
        writer.writerow(["Day", "Teacher", "Class", "Shift", "Subject"])

        current_day_idx = None
        for (teacher, cls, shift, day_idx, day_name, subject) in results:
            # If we've moved to a new day, print the day name
            # Otherwise, leave it blank for a "merged" effect
            if day_idx != current_day_idx:
                current_day_idx = day_idx
                writer.writerow([day_name, teacher, cls, shift, subject])
            else:
                writer.writerow(["", teacher, cls, shift, subject])

    print("Schedule saved to schedule_results.csv")

else:
    print("No solution found!")


Schedule saved to schedule_results.csv


In [254]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    with open("schedule_for_teacher.csv", mode="w", newline="") as file:
        writer = csv.writer(file)

        teacher_schedule_data = {}
        teachers = set() # To keep track of unique teachers
        for (teacher, cls, shift, day_idx, day_name, subject) in results:
            teachers.add(teacher)
            if teacher not in teacher_schedule_data:
                teacher_schedule_data[teacher] = {}
            if shift not in teacher_schedule_data[teacher]:
                teacher_schedule_data[teacher][shift] = {}
            teacher_schedule_data[teacher][shift][day_name] = (cls, subject)

        day_names_order = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
        sorted_teachers = sorted(list(teachers)) # Sort teachers for consistent output

        # Header row: Teacher and Shift columns
        header_row = ["Teacher"]
        for shift in range(num_shifts):
            header_row.append(f"Shift {shift+1}")
        writer.writerow(header_row)

        for teacher in sorted_teachers:
            teacher_row = [teacher]
            for shift in range(num_shifts):
                shift_schedule_list = []
                if teacher in teacher_schedule_data and shift in teacher_schedule_data[teacher]:
                    for day_name in day_names_order:
                        if day_name in teacher_schedule_data[teacher][shift]:
                            cls, subject = teacher_schedule_data[teacher][shift][day_name]
                            shift_schedule_list.append(f"{day_name}: Class {cls}-{subject}")
                shift_schedule_text = "\n".join(shift_schedule_list) # Join days with newline for column format
                teacher_row.append(shift_schedule_text)
            writer.writerow(teacher_row)

    print("Schedule saved to schedule_for_teacher.csv")

else:
    print("No solution found!")

Schedule saved to schedule_for_teacher.csv
